In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # to read records completely
import pickle
import numpy as np
import itertools
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer

## Previous data prep necessary for c) and d)

In [2]:
df = pd.read_pickle('data_clean.pkl')

In [3]:
models = pd.read_csv('models.csv', header=None, names = ['brand', 'model'])
models['brand'] = models['brand'].str.replace(r'[^\w\s]+', '', regex=True) # clean some punctuation errors (nissan.)

In [4]:
models = models[~models.brand.isin(['car', 'sedan', 'problem'])]

In [5]:
df.Message.str.contains('said').sum()

370

In [6]:
df.Message_words = df.Message_words.apply(lambda x: [i.lower() for i in x])

### Replace models with brands

In [7]:
messages = df.Message_words.to_list()

In [8]:
# numpy.where to do the replacement
messages2 = []
t = 0
for m in messages:
    for i,row in models.iterrows():
        m = np.where(m == row['model'], row['brand'], m)
        
    t+=1
    
    #print(t)
        
    messages2.append(m)
        
df['Message_words_v2'] = messages2

In [9]:
# let's find the occurences of changes
m1 = df.Message_words.to_list()
m2 = df.Message_words_v2.to_list()

c = 0
W = 0
for i in range(len(m2)):
    for j in range(len(m2[i])):
        if m1[i][j] != m2[i][j]: #word by word comparison
            c +=1
        W += 1

In [10]:
print(f'{c} changes of models')

6368 changes of models


In [11]:
df.head(10)

,Date,User_Id,Message,NumberOfPastPosts,Role,Message_words,Message_words_v2
0,2001-09-01,pat,"Need help choosing your next vehicle? Tell us your price range, are you thinking New or Used, buying or leasing, what features are must-haves, what other thoughts are on your mind, and let us give you a hand! Tell us your criteria as specifically as you can.You'll find lots of helpful folks here who can give you useful suggestions.","10,421",Member,"[need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]","[need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]"
1,2001-09-01,willow25,"I'm buying a ""new"" car by the end of the year and my first choice is a Nissan Maxima. I've read that they are very reliable and hold their value well. My budget is probably going to allow for a 98-99 and I was wondering if it's worth buying a car that's almost 3-4 years old. Any suggestions or comments would help a lot.willow25",2,Member,"[im, buying, new, car, end, year, first, choice, nissan, maxima, ive, read, reliable, hold, value, well, budget, probably, going, allow, 9899, wondering, worth, buying, car, thats, almost, 34, years, old, suggestions, comments, would, help, lotwillow25]","[im, buying, new, car, end, year, first, choice, nissan, nissan, ive, read, reliable, hold, value, well, budget, probably, going, allow, 9899, wondering, worth, buying, car, thats, almost, 34, years, old, suggestions, comments, would, help, lotwillow25]"
2,2001-10-01,dindak,"Buy a 2001/ 2002 Oldsmobile Intrigue. It's one of the best sedans on the market with great handling and one of the best V6 engines around, a 3.5L DOHC producing 215HP. The deals should be great this year as it's the last on for Intrigue and the warrany on them has been extended to 5 years. The car is recommended by most magazines (including Consumer Reports), but is unfortunately over looked by most consumers. It is the sleeper of all sleepers on the market in my opinion.","6,632",Member,"[buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]","[buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]"
3,2001-10-01,peteri1,"Am looking at 1 of 4 choices to replace my 99GS400. I've read everything here and on other forums and have seen much conflicting info:4 options:Lexus LS430 with custom/luxury packageInfiniti Q45 with premium pckgAudi A-6 4.2 loaded Audi A-8 not stripped, but not loadedChicago winters--had Blizzaks on the GS--worked fine. My 95Q was pig in snow, but I'm sure 2002Q is a world away from that. I've heard Audi has some QC problems, but I like their maintenance being included. I'm a happy Lexus driver. The Q appears to offer the most for the $$. Will lease and in this economy, I would think I could command an attractive package. Can I go wrong with any of the 4? Am I missing something?",21,Member,"[looking, 1, 4, choices, replace, 99gs400, ive, read, everything, forums, seen, much, conflicting, info4, optionslexus, ls430, customluxury, packageinfiniti, q45, premium, pckgaudi, a6, 42, loaded, audi, a8, stripped, loadedchicago, wintersh

In [12]:
#Get a unique list of brands
brands_list = models.brand.drop_duplicates().to_list()

In [13]:
#Removing duplicates in lists while keeping order
#import itertools
def remove_consecutive_duplicate (text):
    return np.array([i for i, j in itertools.groupby(text)])

In [14]:
df["Message_words_v3"] = df["Message_words_v2"].apply(remove_consecutive_duplicate)

In [15]:
type(df["Message_words_v2"][0])

numpy.ndarray

In [16]:
type(df["Message_words_v3"][0])

numpy.ndarray

In [17]:
df.head(10)

,Date,User_Id,Message,NumberOfPastPosts,Role,Message_words,Message_words_v2,Message_words_v3
0,2001-09-01,pat,"Need help choosing your next vehicle? Tell us your price range, are you thinking New or Used, buying or leasing, what features are must-haves, what other thoughts are on your mind, and let us give you a hand! Tell us your criteria as specifically as you can.You'll find lots of helpful folks here who can give you useful suggestions.","10,421",Member,"[need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]","[need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]","[need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]"
1,2001-09-01,willow25,"I'm buying a ""new"" car by the end of the year and my first choice is a Nissan Maxima. I've read that they are very reliable and hold their value well. My budget is probably going to allow for a 98-99 and I was wondering if it's worth buying a car that's almost 3-4 years old. Any suggestions or comments would help a lot.willow25",2,Member,"[im, buying, new, car, end, year, first, choice, nissan, maxima, ive, read, reliable, hold, value, well, budget, probably, going, allow, 9899, wondering, worth, buying, car, thats, almost, 34, years, old, suggestions, comments, would, help, lotwillow25]","[im, buying, new, car, end, year, first, choice, nissan, nissan, ive, read, reliable, hold, value, well, budget, probably, going, allow, 9899, wondering, worth, buying, car, thats, almost, 34, years, old, suggestions, comments, would, help, lotwillow25]","[im, buying, new, car, end, year, first, choice, nissan, ive, read, reliable, hold, value, well, budget, probably, going, allow, 9899, wondering, worth, buying, car, thats, almost, 34, years, old, suggestions, comments, would, help, lotwillow25]"
2,2001-10-01,dindak,"Buy a 2001/ 2002 Oldsmobile Intrigue. It's one of the best sedans on the market with great handling and one of the best V6 engines around, a 3.5L DOHC producing 215HP. The deals should be great this year as it's the last on for Intrigue and the warrany on them has been extended to 5 years. The car is recommended by most magazines (including Consumer Reports), but is unfortunately over looked by most consumers. It is the sleeper of all sleepers on the market in my opinion.","6,632",Member,"[buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]","[buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]","[buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]"
3,2001-10-01,peteri1,"Am looking at 1 of 4 choices to replace my 99GS400. I've read everything here

In [18]:
#Removing duplicate brands in lists but only

## Link attributes to models

In [19]:
# first lets count brand mentions in each record

In [20]:
messages = df.Message_words_v3.to_list()

In [21]:
brands_in_message = []
for m in messages:
    
    brands_mentioned = []
    for brand in brands_list:
        if len(np.where(m == brand)[0]) >0:
            brands_mentioned.append(brand)
            
    brands_in_message.append(brands_mentioned)
        
    

C:\Users\louis\AppData\Local\Temp/ipykernel_17764/3185542915.py:6: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if len(np.where(m == brand)[0]) >0:


In [22]:
df['Brands_in_message'] = brands_in_message

In [23]:
df['Qty_brands'] = df.Brands_in_message.apply(lambda x: len(x))

In [24]:
df['Qty_brands'].value_counts()

# Most messages discuss a single brand
# Also common to compare 2 brands
# 727 messages don't mention any brand (remove them from analysis?)
# Passing a certain treshold (5+), I think it would be better to remove comments since it will be hard to figure out which
# attribute belongs to each brand

0     1291
1     1215
2      755
3      459
4      219
5      109
6       54
7       24
8       10
9        6
11       4
10       3
16       1
13       1
Name: Qty_brands, dtype: int64

In [25]:
df.loc[df.Qty_brands == 0, 'Message_words_v3'].head(10) # to review records with no brand mentions
# most of them are for seeking advice

0                                                                                       [need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]
2     [buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]
8                                                                                                                                                                                                                                                                                      [id, pass, intrigue, one, reaso

In [26]:
m = df.Message_words_v3[9] #record 9 is a good exaple
brand_dic = {}
for brand in brands_list:
    ix = np.where(m == brand)[0]
    if len(ix) > 0:
        brand_dic[brand] = ix[0]
        

brand_dic = dict(sorted(brand_dic.items(), key=lambda x:x[1]))

brand_list = list(brand_dic.values()) + [len(m)]

for i, tup in enumerate(brand_dic):
    print(brand_dic[tup])
    print(tup)
    
    if i == 0:
        
        print(m[0:brand_list[1]])
    else:
        print(m[brand_list[i]:brand_list[i+1]])
        
        

2
honda
['would' 'recommend' 'honda']
3
toyota
['toyota' 'choices' 'sedan' 'new']
7
nissan
['nissan' 'looks' 'impressive' 'would' 'wait' 'reliability' '2002'
 'proventhe' 'new']
16
bmw
['bmw' '525' 'another' 'nice' 'car' 'rated' 'highly' 'many' 'auto'
 'publications']


In [27]:
# now we run it for all records

brand_reviews = []

for m in df.Message_words_v3:

    brand_dic = {}
    for brand in brands_list:
        ix = np.where(m == brand)[0]
        if len(ix) > 0:
            brand_dic[brand] = ix[0]

    brand_dic = dict(sorted(brand_dic.items(), key=lambda x:x[1]))

    brand_list = list(brand_dic.values()) + [len(m)]


    review={}
    for i, tup in enumerate(brand_dic):
        #print(brand_dic[tup])
        #print(tup)

        if i == 0:
            
            review[tup] = m[0:brand_list[1]]
        else:
            review[tup] = m[brand_list[i]:brand_list[i+1]]
            
    brand_reviews.append(review)

# generates a list (1 entry per row) of dictionaries {brand: part of text corresponding}

C:\Users\louis\AppData\Local\Temp/ipykernel_17764/721950690.py:9: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  ix = np.where(m == brand)[0]


In [28]:
len(brand_reviews) # each record becomes a single dictionary with all its brand mentions and linked words

4151

In [29]:
brand_reviews[9]

{'honda': array(['would', 'recommend', 'honda'], dtype='<U12'),
 'toyota': array(['toyota', 'choices', 'sedan', 'new'], dtype='<U12'),
 'nissan': array(['nissan', 'looks', 'impressive', 'would', 'wait', 'reliability',
        '2002', 'proventhe', 'new'], dtype='<U12'),
 'bmw': array(['bmw', '525', 'another', 'nice', 'car', 'rated', 'highly', 'many',
        'auto', 'publications'], dtype='<U12')}

In [30]:
# now we need to concatenate all reviews for each brand into a single key in a dictionary
review_accum = {i:[] for i in brands_list}

In [31]:
for brand in brands_list:
    for review in brand_reviews:
        for single_review in review:
            #print(review[single_review])
            if single_review==brand:
                review_accum[brand].extend(review[single_review])

In [32]:
## from here, we are ready to start counting attributes per brand
## we also need to consider replacing some similar attributes before doing the final count 
## I will do one example with Audi but it could become a function and run in a loop to all the brands 

In [33]:
# first lets discover popular attributes regardless of brands

all_attributes = []

for i in list(review_accum.values()):
    all_attributes.extend(i)
    
words = pd.Series(all_attributes)

words.value_counts().head(40) # this shows the frequencies of each word
# the most "rustic" approach would be to literally print them in order and take note of the ones that can be considered "attributes"

car           3152
honda         1746
like          1634
would         1600
get           1453
new           1394
toyota        1341
cars          1324
good          1249
one           1178
also           964
nissan         963
im             943
dont           883
drive          880
miles          823
much           808
want           802
better         796
years          776
used           766
think          750
volkswagen     738
price          737
looking        714
well           699
really         669
buy            583
know           579
go             560
even           560
chevrolet      556
seat           551
need           550
great          533
year           532
could          529
ive            495
driving        487
might          481
dtype: int64

In [34]:
words[~words.isin(brands_list)].value_counts().head(60)


car            3152
like           1634
would          1600
get            1453
new            1394
cars           1324
good           1249
one            1178
also            964
im              943
dont            883
drive           880
miles           823
much            808
want            802
better          796
years           776
used            766
think           750
price           737
looking         714
well            699
really          669
buy             583
know            579
even            560
go              560
need            550
great           533
year            532
could           529
ive             495
driving         487
might           481
time            479
look            466
lot             456
best            454
engine          454
something       446
3               445
reliability     442
id              441
nice            439
going           434
vehicle         433
civic           431
old             424
probably        419
still           409


In [80]:
words_count = words.value_counts()
words_nobrands_count = words[~words.isin(brands_list)].value_counts()

### Stemming

In [36]:
#Stemming
stemmer = PorterStemmer()

In [37]:
def stem_list (text):
    return pd.Series([stemmer.stem(w) for w in text])

In [38]:
stemmed_words = stem_list(words)
len(stemmed_words)

191416

In [39]:
stemmed_words.value_counts().head(60)

car           4477
like          1924
get           1854
honda         1746
would         1600
look          1585
drive         1477
new           1394
toyota        1341
year          1308
one           1270
good          1251
use           1023
go             994
want           993
also           964
nissan         963
im             943
price          925
mile           900
think          890
dont           883
buy            848
much           808
reliabl        806
better         799
volkswagen     741
need           738
well           699
vehicl         690
realli         669
know           635
model          633
time           589
seat           581
make           581
lot            578
even           562
chevrolet      558
engin          557
great          533
could          529
dealer         506
ive            495
consid         485
might          481
sedan          463
nice           459
civic          459
best           455
someth         446
3              445
audi        

In [40]:
stemmed_words[~stemmed_words.isin(brands_list)].value_counts().head(60)


car        4477
like       1924
get        1854
would      1600
look       1585
drive      1477
new        1394
year       1308
one        1270
good       1251
use        1023
go          994
want        993
also        964
im          943
price       925
mile        900
think       890
dont        883
buy         848
much        808
reliabl     806
better      799
need        738
well        699
vehicl      690
realli      669
know        635
model       633
time        589
make        581
lot         578
even        562
engin       557
great       533
could       529
dealer      506
ive         495
consid      485
might       481
sedan       463
nice        459
civic       459
best        455
someth      446
3           445
id          442
leas        442
old         441
find        422
option      422
probabl     420
test        419
seem        415
back        409
4           409
still       409
cost        403
less        402
around      396
dtype: int64

In [41]:
stemwords_count = stemmed_words.value_counts()
stemwords_nobrands_count = stemmed_words[~stemmed_words.isin(brands_list)].value_counts()

In [77]:
type(stemwords_count)

pandas.core.series.Series

In [42]:
#Export value count series to a single multi-sheet Excel
#pip install xlsxwriter
writer = pd.ExcelWriter('attribute_count.xlsx', engine='xlsxwriter')


In [43]:
words_count.to_excel(writer, sheet_name='words_valuecounts')
words_nobrands_count.to_excel(writer, sheet_name='words_nobrands_valuecounts')
stemwords_count.to_excel(writer, sheet_name='stemwords_valuecounts')
stemwords_nobrands_count.to_excel(writer, sheet_name='stemwords_nobrands_valuecounts')

In [44]:
writer.save()


In [82]:
#For words_nobrands_count remove words that appear 2x or less
words_nobrands_count = words_nobrands_count[words_nobrands_count > 2]
words_nobrands_count

car         3152
like        1634
would       1600
get         1453
new         1394
            ... 
nope           3
scrape         3
850            3
modelsi        3
rearview       3
Length: 5361, dtype: int64

In [83]:
words_nobrands_count.to_csv("attribute_count_final.csv")

### TF-IDF

In [53]:
df

,Date,User_Id,Message,NumberOfPastPosts,Role,Message_words,Message_words_v2,Message_words_v3,Brands_in_message,Qty_brands
0,2001-09-01,pat,"Need help choosing your next vehicle? Tell us your price range, are you thinking New or Used, buying or leasing, what features are must-haves, what other thoughts are on your mind, and let us give you a hand! Tell us your criteria as specifically as you can.You'll find lots of helpful folks here who can give you useful suggestions.","10,421",Member,"[need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]","[need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]","[need, help, choosing, next, vehicle, tell, us, price, range, thinking, new, used, buying, leasing, features, musthaves, thoughts, mind, let, us, give, hand, tell, us, criteria, specifically, canyoull, find, lots, helpful, folks, give, useful, suggestions]",[],0
1,2001-09-01,willow25,"I'm buying a ""new"" car by the end of the year and my first choice is a Nissan Maxima. I've read that they are very reliable and hold their value well. My budget is probably going to allow for a 98-99 and I was wondering if it's worth buying a car that's almost 3-4 years old. Any suggestions or comments would help a lot.willow25",2,Member,"[im, buying, new, car, end, year, first, choice, nissan, maxima, ive, read, reliable, hold, value, well, budget, probably, going, allow, 9899, wondering, worth, buying, car, thats, almost, 34, years, old, suggestions, comments, would, help, lotwillow25]","[im, buying, new, car, end, year, first, choice, nissan, nissan, ive, read, reliable, hold, value, well, budget, probably, going, allow, 9899, wondering, worth, buying, car, thats, almost, 34, years, old, suggestions, comments, would, help, lotwillow25]","[im, buying, new, car, end, year, first, choice, nissan, ive, read, reliable, hold, value, well, budget, probably, going, allow, 9899, wondering, worth, buying, car, thats, almost, 34, years, old, suggestions, comments, would, help, lotwillow25]",[nissan],1
2,2001-10-01,dindak,"Buy a 2001/ 2002 Oldsmobile Intrigue. It's one of the best sedans on the market with great handling and one of the best V6 engines around, a 3.5L DOHC producing 215HP. The deals should be great this year as it's the last on for Intrigue and the warrany on them has been extended to 5 years. The car is recommended by most magazines (including Consumer Reports), but is unfortunately over looked by most consumers. It is the sleeper of all sleepers on the market in my opinion.","6,632",Member,"[buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]","[buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]","[buy, 2001, 2002, oldsmobile, intrigue, one, best, sedans, market, great, handling, one, best, v6, engines, around, 35l, dohc, producing, 215hp, deals, great, year, last, intrigue, warrany, extended, 5, years, car, recommended, magazines, including, consumer, reports, unfortunately, looked, consumers, sleeper, sleepers, market, opinion]",[],0
3,2001-10-01,peteri1,"Am looking at 1 of 4 choice

In [61]:
def listtostring(text):
    return " ".join(text) 

In [63]:
df["Message_words_v4"] = df["Message_words_v3"].apply(listtostring)

In [66]:
vectorizer = TfidfVectorizer()

In [69]:
X = vectorizer.fit_transform(df["Message_words_v4"].tolist())

In [71]:
df_tfidf = pd.DataFrame(X[0].T.todense(), index=vectorizer.get_feature_names_out(), columns=["TF-IDF"])
df_tfidf = df_tfidf.sort_values('TF-IDF', ascending=False)

In [73]:
df_tfidf.to_csv("attribute_tfidf.csv")

In [45]:
# once we have the list of the 5 most popular attributes, my guess is we have to calculate the uplift scores of said attribute
# against the 5 top brands found in letter A). Then the uplift should tell us which brand is more related to which attribute

In [46]:
# let's say "reliability" is one of the popular attributes 
# and the top 5 brands are: honda, nissan, audi, bwm, toyota
# let's get the inputs necessary to calculate uplift of reliability in nissan

In [47]:
n=0
for i in  df.Brands_in_message:
    if 'nissan' in i:
        n += 1
n
#nissan appears in 525 rows 

525

In [48]:
df_nissan = pd.DataFrame(review_accum['nissan'], columns = ['word'])
(df_nissan.word == 'reliability').sum() # 73 mentions of "relibaility" in honda

# and 28 times the term "reliability" was mentioned alongside a nissan

28

In [49]:
# how about honda?

n=0
for i in  df.Brands_in_message:
    if 'honda' in i:
        n += 1
print(n)


df_honda = pd.DataFrame(review_accum['honda'], columns = ['word'])
(df_honda.word == 'reliability').sum() # 73 mentions of "relibaility" in honda


924


73

In [50]:
73/924 # not the exact uplift formula but a quick validation

0.07900432900432901

In [51]:
28/525 # vs nissan

0.05333333333333334

In [52]:
# honda has more prevalence as a reliable brand than nissan (pending to adjust for uplift real formula)

to consider:

right now the code counts all the appearances of an attribute, even if they come from the same record; I think that is not the exact definition of the uplift formula. I don't think it will change the numbers very much, but would be good to refine it if we have the time